# Demostración del Modelo de Simplificación de Textos Médicos

Este notebook muestra cómo utilizar el modelo de IA para simplificar textos médicos técnicos en explicaciones claras y comprensibles para pacientes.

## Contenido
1. Configuración del entorno
2. Carga del modelo
3. Ejemplos de uso
4. Visualización de resultados
5. Demo interactiva

In [ ]:
# Importar las bibliotecas necesarias
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

## Carga del Modelo y Tokenizer

Cargaremos el modelo base FLAN-T5 con los adaptadores LoRA entrenados para la simplificación de textos médicos.

In [ ]:
# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained("../model")

# Cargar modelo base
modelo_base = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-large",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
)

# Verificar y ajustar el tamaño del vocabulario
target_vocab_size = len(tokenizer)
current_vocab = modelo_base.get_input_embeddings().weight.shape[0]
if current_vocab != target_vocab_size:
    print(f"Redimensionando embeddings: {current_vocab} -> {target_vocab_size}")
    modelo_base.resize_token_embeddings(target_vocab_size)

# Cargar adaptadores LoRA
modelo = PeftModel.from_pretrained(modelo_base, "../model")
modelo.eval()

# Mover a GPU si está disponible
if torch.cuda.is_available():
    modelo = modelo.to("cuda:0")
    print("Modelo cargado en GPU")
else:
    print("Modelo cargado en CPU")

## Función de Inferencia

Definiremos una función que simplifica el texto médico usando nuestro modelo.

In [ ]:
def simplificar_texto_medico(texto):
    """
    Simplifica un texto médico técnico en una explicación clara para pacientes.
    
    Args:
        texto (str): Texto médico técnico a simplificar
        
    Returns:
        str: Texto simplificado
    """
    # Preparar el prompt
    prompt = f"Transforma el siguiente texto médico técnico en una explicación clara, empática y comprensible para pacientes. Usa lenguaje sencillo, evita términos especializados, y transmite la información de forma neutral y accesible: {texto}"
    
    # Tokenizar
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=128)
    if torch.cuda.is_available():
        inputs = inputs.to("cuda:0")
    
    # Generar respuesta
    with torch.no_grad():
        outputs = modelo.generate(
            **inputs,
            max_new_tokens=64,
            num_beams=4,
            pad_token_id=tokenizer.pad_token_id
        )
    
    # Decodificar y retornar
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## Demo Interactiva

Crearemos una interfaz interactiva para probar el modelo con diferentes textos médicos.

In [ ]:
def crear_interfaz_demo():
    # Crear widgets
    texto_input = widgets.Textarea(
        value='',
        placeholder='Ingrese el texto médico aquí...',
        description='Texto:',
        disabled=False,
        layout={'width': '100%', 'height': '100px'}
    )
    
    boton_simplificar = widgets.Button(
        description='Simplificar',
        button_style='primary',
        tooltip='Haz clic para simplificar el texto'
    )
    
    output = widgets.Output()
    
    # Función de callback
    def on_button_click(b):
        with output:
            output.clear_output()
            if not texto_input.value:
                print("Por favor, ingrese un texto médico para simplificar.")
                return
                
            print("Texto original:")
            print(texto_input.value)
            print("\nTexto simplificado:")
            print(simplificar_texto_medico(texto_input.value))
    
    boton_simplificar.on_click(on_button_click)
    
    # Mostrar widgets
    display(texto_input)
    display(boton_simplificar)
    display(output)

# Crear la interfaz
crear_interfaz_demo()

## Ejemplos de Uso

A continuación se muestran algunos ejemplos de textos médicos y sus simplificaciones:

In [ ]:
# Lista de ejemplos
ejemplos = [
    "Paciente con hiperlipidemia mixta sin tratamiento previo",
    "Síndrome de apnea-hipopnea obstructiva del sueño severa sin tratamiento",
    "Crisis hipertensiva con edema agudo de pulmón"
]

# Procesar y mostrar cada ejemplo
for texto in ejemplos:
    print("Texto original:")
    print(texto)
    print("\nTexto simplificado:")
    print(simplificar_texto_medico(texto))
    print("\n" + "-"*50 + "\n")